In [2]:
!pip install PyQt5
import sys
import numpy as np
import pickle as pk
from PyQt5.QtWidgets import (
    QApplication, QWidget, QLabel, QLineEdit, QPushButton,
    QVBoxLayout, QHBoxLayout, QComboBox, QMessageBox
)
from PyQt5.QtCore import Qt

# Cargar el modelo
with open("modelo_final.pickle", 'rb') as f:
    modelo = pk.load(f)

class WhiskyPredictor(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Predicción de Whisky")
        self.init_ui()

    def init_ui(self):
        layout = QVBoxLayout()

        # Campos de entrada
        self.inputs = {}
        campos = [
            ("Fecha (YYYYMMDD):", "date"),
            ("Número de tienda:", "store_number"),
            ("Código postal (ZIP):", "zip_code"),
            ("Número de condado:", "county_number"),
            ("Número de botellas:", "pack"),
            ("Volumen botella (ml):", "bottle_volume_ml"),
            ("Botellas vendidas:", "bottles_sold"),
            ("Número de vendedor:", "vendor_number"),
            ("Costo de la botella por Estado:", "state_bottle_cost")
        ]

        for label_text, key in campos:
            h_layout = QHBoxLayout()
            label = QLabel(label_text)
            entry = QLineEdit()
            entry.textChanged.connect(self.validar_campos)
            h_layout.addWidget(label)
            h_layout.addWidget(entry)
            layout.addLayout(h_layout)
            self.inputs[key] = entry

        # Selector de whisky
        h_layout = QHBoxLayout()
        h_layout.addWidget(QLabel("Tipo de Whisky:"))
        self.whisky_combo = QComboBox()
        self.whisky_combo.addItems([
            "", "Category_BLENDED WHISKIES", "Category_CANADIAN WHISKIES",
            "Category_IRISH WHISKIES ", "Category_SCOTCH WHISKIES",
            "Category_STRAIGHT BOURBON WHISKIES", "Category_TENNESSEE WHISKIES"
        ])
        self.whisky_combo.currentTextChanged.connect(self.validar_campos)
        h_layout.addWidget(self.whisky_combo)
        layout.addLayout(h_layout)

        # Botón
        self.btn_pred = QPushButton("Hacer Predicción")
        self.btn_pred.setEnabled(False)
        self.btn_pred.clicked.connect(self.hacer_prediccion)
        layout.addWidget(self.btn_pred)

        self.setLayout(layout)

    def validar_campos(self):
        try:
            date = int(self.inputs["date"].text())
            if not (2000101 <= date <= 20241231):
                raise ValueError("Fecha inválida")

            zip_code = int(self.inputs["zip_code"].text())
            if not (50001 <= zip_code <= 52809):
                raise ValueError("ZIP inválido")

            county_number = float(self.inputs["county_number"].text())
            if not (1 <= county_number <= 99):
                raise ValueError("Condado inválido")

            store_number = int(self.inputs["store_number"].text())
            pack = int(self.inputs["pack"].text())
            bottle_volume = int(self.inputs["bottle_volume_ml"].text())
            bottles_sold = int(self.inputs["bottles_sold"].text())
            vendor_number = float(self.inputs["vendor_number"].text())
            bottle_cost = float(self.inputs["state_bottle_cost"].text())

            if any(x <= 0 for x in [store_number, pack, bottle_volume, bottles_sold, vendor_number]):
                raise ValueError("Valores deben ser > 0")
            if bottle_cost == 0:
                raise ValueError("Costo no puede ser cero")

            if self.whisky_combo.currentText() == "":
                raise ValueError("Tipo de whisky no seleccionado")

            self.btn_pred.setEnabled(True)
        except:
            self.btn_pred.setEnabled(False)

    def hacer_prediccion(self):
        try:
            entradas = [
                int(self.inputs["date"].text()),
                int(self.inputs["store_number"].text()),
                int(self.inputs["zip_code"].text()),
                float(self.inputs["county_number"].text()),
                int(self.inputs["pack"].text()),
                int(self.inputs["bottle_volume_ml"].text()),
                int(self.inputs["bottles_sold"].text()),
                float(self.inputs["vendor_number"].text()),
                float(self.inputs["state_bottle_cost"].text())
            ]

            tipo = self.whisky_combo.currentText()
            categorias = [
                "Category_BLENDED WHISKIES",
                "Category_CANADIAN WHISKIES",
                "Category_IRISH WHISKIES ",
                "Category_SCOTCH WHISKIES",
                "Category_STRAIGHT BOURBON WHISKIES",
                "Category_TENNESSEE WHISKIES"
            ]
            entradas.extend([tipo == cat for cat in categorias])

            pred = modelo.predict(np.array(entradas).reshape(1, -1))
            QMessageBox.information(self, "Resultado", f"La predicción del precio de venta es: {pred[0]:.2f} dolares")

        except Exception as e:
            QMessageBox.critical(self, "Error", f"Error durante la predicción: {str(e)}")

# Lanzar aplicación
if __name__ == "__main__":
    app = QApplication(sys.argv)
    ventana = WhiskyPredictor()
    ventana.show()
    sys.exit(app.exec_())


   ---------------------------------------- 0.0/6.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.9 MB ? eta -:--:--
   - -------------------------------------- 0.3/6.9 MB ? eta -:--:--
   --- ------------------------------------ 0.5/6.9 MB 932.9 kB/s eta 0:00:07
   ---- ----------------------------------- 0.8/6.9 MB 1.4 MB/s eta 0:00:05
   ------ --------------------------------- 1.0/6.9 MB 1.1 MB/s eta 0:00:06
   ------ --------------------------------- 1.0/6.9 MB 1.1 MB/s eta 0:00:06
   ------- -------------------------------- 1.3/6.9 MB 1.1 MB/s eta 0:00:06
   --------- ------------------------------ 1.6/6.9 MB 1.0 MB/s eta 0:00:06
   --------- ------------------------------ 1.6/6.9 MB 1.0 MB/s eta 0:00:06
   --------- ------------------------------ 1.6/6.9 MB 1.0 MB/s eta 0:00:06
   ---------- ----------------------------- 1.8/6.9 MB 818.3 kB/s eta 0:00:07
   ---------- ----------------------------- 1.8/6.9 MB 818.3 kB/s eta 0:00:07
   ------------ -----------


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
c:\Users\ASUS\miniconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


SystemExit: 0

c:\Users\ASUS\miniconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
